In [32]:
%matplotlib inline
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from datetime import datetime, timedelta
import math
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import gridspec
from datetime import datetime
import numpy as np
import pandas as pd
import csv

es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)
my_index = ["ps_trace-2018.2*", "ps_packetloss-2018.2*"]

my_query = {}

# CHANGE SO IT RETREIVES ONLY UNIQUE hashes.

# sS='UC'
# srcSiteOWDServer = "192.170.227.160"
# srcSiteThroughputServer = "192.170.227.162"

sS='CERN-PROD'
srcSiteOWDServer = "128.142.223.247"
srcSiteThroughputServer = "128.142.223.246"

# dS='IU'
# destSiteOWDServer = "149.165.225.223"
# destSiteThroughputServer = "149.165.225.224"

# dS='UIUC'
# destSiteOWDServer = "72.36.96.4"
# destSiteThroughputServer = "72.36.96.9"

# dS='ICCN'
# destSiteOWDServer = "72.36.96.4"
# destSiteThroughputServer = "72.36.126.132"

dS='pic'
destSiteOWDServer = "193.109.172.188"
destSiteThroughputServer = "193.109.172.187"
'''
#grab user system current date
current_date = datetime.today()

#correct 1 digit days/months to DD/MM format
user_yr = str(current_date.year)
user_month = str(current_date.month)
if len(user_month) < 2:
    user_month = '0' + user_month
user_day = str(current_date.day)
if len(user_day) < 2:
    user_day = '0' + user_day
end_date = user_yr + user_month + user_day + 'T000000Z'

#subtract 1 week from current date
begin_raw = current_date - timedelta(weeks=10)

#do same corrections as above
start_yr = str(begin_raw.year)
start_month = str(begin_raw.month)
if len(start_month) < 2:
    start_month = '0' + start_month
start_day = str(begin_raw.day)
if len(start_day) < 2:
    start_day = '0' + start_day
start_date = start_yr + start_month + start_day + 'T000000Z'
'''

start_date = '20180215T000000Z'
end_date = '20180215T002059Z'

my_query = {
    "size":1,
    "_source": {
        "include": [ 'src','dest','hops','hash','packet_loss' ]
    },
    'query':{
        'bool':{
            'must':[
                {'range': {'timestamp': {'gte': start_date, 'lt': end_date}}},
#                {'term': {'_type': 'traceroute'}},
#                         {'bool':
#                             {'should':[
#                                 {'term': {'src': srcSiteOWDServer}},
#                                 {'term': {'src': srcSiteThroughputServer}},
#                                 {'term': {'src': destSiteOWDServer}},
#                                 {'term': {'src': destSiteThroughputServer}}
#                             ]}
#                         }
#                         ,
#                         {'bool':
#                             {'should':[
#                                 {'term': {'dest': destSiteOWDServer}},
#                                 {'term': {'dest': destSiteThroughputServer}},
#                                 {'term': {'dest': srcSiteOWDServer}},
#                                 {'term': {'dest': srcSiteThroughputServer}}
#                             ]}
#                         }
            ]

        }
    },
    "aggs": {
        "grouped_by_hash": {
          "terms": {  "field": "hash", "size":1000 }, #
          "aggs": {
              "top_hash_hits": {
                  "top_hits": {
                      "sort": [ { "_score": { "order": "desc" } } ],
                      "size": 1
                  }
              }
          }
       }
    }
}
results = es.search(body=my_query, index=my_index, request_timeout=10000)
#results2 = es.search(body=my_query, index=my_index[1], request_timeout=10000)
print(results['aggregations']['grouped_by_hash']['buckets'])
#print("whuhwwhhwfhiw")
#print(results)
#print("IWFUQG8FOWHIFHOIWFQOF")
#print(results2)
print('done')

[{'key': 3161852279277434577, 'top_hash_hits': {'hits': {'hits': [{'_source': {'src_VO': 'ATLAS', 'dest_host': 'perfmon.dur.scotgrid.ac.uk', 'hops': ['128.40.4.245', '128.40.20.149'], 'timestamp': 1518653251000, 'src_site': 'UKI-LT2-UCL-HEP', 'dest_site': 'UKI-SCOTGRID-DURHAM', 'n_hops': 2, 'MA': '193.60.193.3', 'dest_production': True, 'max_rtt': 0.2, 'src_production': True, 'src': '128.40.4.25', 'ttls': [1, 2], 'hash': 3161852279277434577, 'dest': '193.60.193.3', 'src_host': 'lcg-sonar01.hep.ucl.ac.uk', 'rtts': [0.2, 0.2], 'dest_VO': 'ATLAS', 'ipv6': False}, '_id': 'AWGWzEnJKTZJJXLfAPp8', '_score': 1.0, '_index': 'ps_trace-2018.2.15', '_type': 'doc'}], 'total': 60, 'max_score': 1.0}}, 'doc_count': 60}, {'key': -8092498709185128013, 'top_hash_hits': {'hits': {'hits': [{'_source': {'dest_host': 'psonartest1.fnal.gov', 'rtts': [0.3], 'timestamp': 1518653406000, 'src_host': 'perfsonar-1.t2.ucsd.edu', 'n_hops': 1, 'MA': '2607:f720:1700:1b30::29', 'dest_production': False, 'max_rtt': 0.3, 

In [3]:
data_size = len(results['aggregations']['grouped_by_hash']['buckets'])
src_to_dest = {}

for i in range(0, data_size):
    rt_src = results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['src']
    rt_src_host = results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['src_host']
    rt_dest = results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['dest']
    rt_dest_host = results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['dest_host']
    rt_hops = results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['hops']
    dupe_rt = 0
    #if rt_src_host == 'perfson2.zeuthen.desy.de':
        #print(rt_src)
    if rt_src in src_to_dest.keys():
        if rt_dest in src_to_dest[rt_src].keys():
            current_rt = 'rt1'
            for x in range(0, len(src_to_dest[rt_src][rt_dest])):
                if src_to_dest[rt_src][rt_dest][current_rt]['hop_list'] == rt_hops:
                    src_to_dest[rt_src][rt_dest][current_rt]['count'] += 1
                    #print('recurring route',rt_src_host,' to ',rt_dest_host)
                    dupe_rt = 1
                current_rt = 'rt' + str(x+2)
            if (dupe_rt == 0):
                src_to_dest[rt_src][rt_dest][current_rt] = {}
                src_to_dest[rt_src][rt_dest][current_rt]['count'] = 1
                src_to_dest[rt_src][rt_dest][current_rt]['hop_list'] = rt_hops
                #print('new route',rt_src_host,' to ',rt_dest_host)
        else:
            src_to_dest[rt_src][rt_dest] = {'rt1':{}}
            src_to_dest[rt_src][rt_dest]['rt1']['count'] = 1
            src_to_dest[rt_src][rt_dest]['rt1']['hop_list'] = rt_hops
            #print('new route',rt_src_host,' to ',rt_dest_host)
        dupe_rt = 0
    #completely new source
    else:
        src_to_dest[rt_src] = {rt_dest:{'rt1':{}}}
        src_to_dest[rt_src][rt_dest]['rt1']['count'] = 1
        src_to_dest[rt_src][rt_dest]['rt1']['hop_list'] = rt_hops      
        #print('new route',rt_src_host,' to ',rt_dest_host)
        
print('ran')

ran


In [6]:
rt_count = 0
for src_key in src_to_dest:
    for dest_key in src_to_dest[src_key]:
        for rt_num in src_to_dest[src_key][dest_key]:
            rt_count += src_to_dest[src_key][dest_key][rt_num]['count']

print(rt_count)

4699


In [7]:
otherthan1 = 0

for src_key in src_to_dest:
    for dest_key in src_to_dest[src_key]:
        for rt_num in src_to_dest[src_key][dest_key]:
            if src_to_dest[src_key][dest_key][rt_num]['count'] > 1:
                otherthan1 += 1
                
print(otherthan1)

5


In [8]:
diffrts = 0

for src_key in src_to_dest:
    for dest_key in src_to_dest[src_key]:
        for rt_num in src_to_dest[src_key][dest_key]:
            if rt_num != 'rt1':
                diffrts += 1
                
print(diffrts)

595


In [9]:
print(results['aggregations']['grouped_by_hash']['buckets'][0])

{'doc_count': 19, 'top_hash_hits': {'hits': {'hits': [{'_source': {'srcVO': 'ATLAS', 'dest_host': 'perfsonar02.cmsaf.mit.edu', 'ttls': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'hops': ['200.136.80.1', '143.108.254.241', '198.32.252.229', '190.103.185.145', '162.252.70.42', '198.71.45.7', '162.252.70.139', '192.5.89.221', '192.5.89.22', '18.32.4.110'], 'MA': '18.12.1.172', 'rtts': [0.884, 7.8180000000000005, 109.41, 142.27200000000002, 123.612, 135.92000000000002, 141.641, 145.671, 145.69099999999997, 145.345], 'srcSite': 'SPRACE', 'srcProduction': True, 'destProduction': True, 'max_rtt': 145.69099999999997, 'src_host': 'perfsonar-bw.sprace.org.br', 'n_hops': 10, 'ipv6': False, 'dest': '18.12.1.172', 'src': '200.136.80.19', 'timestamp': 1510704564000, 'hash': 3154126099007176829}, '_type': 'traceroute', '_score': 2.0, '_index': 'network_weather-2017.11.15', '_id': 'AV-9BXCskCHFrlE6JzVc'}], 'max_score': 2.0, 'total': 19}}, 'key': 3154126099007176829}


In [10]:
print(src_to_dest['141.34.200.29'])

{'192.135.14.39': {'rt1': {'hop_list': ['141.34.200.1', '141.34.249.210', '141.34.249.217', '141.34.249.33', '188.1.153.5', '188.1.144.133', '62.40.126.184', '62.40.126.186', '62.40.126.202', '193.206.129.149', '193.206.129.150', '192.135.14.39'], 'count': 1}}, '90.147.67.253': {'rt1': {'hop_list': ['141.34.200.1', '141.34.249.210', '141.34.249.217', '141.34.249.33', '188.1.153.5', '188.1.144.133', '62.40.126.184', '62.40.126.186', '62.40.126.202', '90.147.80.10', '90.147.80.18', '90.147.80.30', '193.206.130.13', '193.206.130.14', '90.147.67.253'], 'count': 1}}, '149.165.225.224': {'rt1': {'hop_list': ['141.34.200.1', '141.34.249.210', '141.34.249.225', '141.34.249.21', '188.1.153.13', '62.40.126.168', '62.40.98.68', '62.40.126.154', '62.40.126.149', '198.124.80.97', '198.124.80.98', '149.165.225.224'], 'count': 1}}, '192.108.47.6': {'rt1': {'hop_list': ['141.34.200.1', '141.34.249.210', '141.34.249.225', '141.34.249.21', '188.1.153.13', '188.1.153.2', '192.108.46.137', '192.108.46.214

In [12]:
diffrts = 0


for dest_key in src_to_dest['141.34.200.29']:
    for rt_num in src_to_dest['141.34.200.29'][dest_key]:
        diffrts += 1
                
print(diffrts)

34


In [15]:
diffrts = 0


for dest_key in src_to_dest['134.158.123.184']:
    for rt_num in src_to_dest['134.158.123.184'][dest_key]:
        diffrts += 1
                
print(diffrts)

80


In [16]:
diffrts = 0


for dest_key in src_to_dest['204.99.128.11']:
    for rt_num in src_to_dest['204.99.128.11'][dest_key]:
        diffrts += 1
                
print(diffrts)

1


In [14]:
print(src_to_dest)

{'134.158.123.184': {'193.231.25.238': {'rt1': {'hop_list': ['134.158.120.1', '193.51.191.78', '62.40.126.221', '62.40.98.109', '62.40.98.46', '62.40.112.194', '62.40.126.245', '37.128.239.141', '37.128.239.110', '37.128.233.150', '217.73.171.114', '193.231.25.238'], 'count': 1}}, '142.150.19.60': {'rt1': {'hop_list': ['134.158.120.1', '193.51.191.78', '62.40.126.221', '62.40.126.173', '205.189.32.76', '205.189.32.192', '206.12.9.89', '128.100.200.198', '142.150.19.60'], 'count': 1}}, '129.215.213.69': {'rt1': {'hop_list': ['134.158.120.1', None, '193.51.180.38', '62.40.124.61', '62.40.98.152', '62.40.98.76', '62.40.98.78', '62.40.124.198', '146.97.33.2', '146.97.33.22', '146.97.33.42', '146.97.36.222', '146.97.71.126', '146.97.41.166', '129.215.213.69'], 'count': 1}}, '149.165.225.224': {'rt1': {'hop_list': ['134.158.120.1', '193.51.191.78', '62.40.126.221', '62.40.126.153', '198.124.80.97', '198.124.80.98', None, None, '149.165.225.224'], 'count': 1}}, '192.108.47.6': {'rt1': {'hop_l